In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#install  +import


In [4]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!pip install bert-sklearn/.

fatal: destination path 'bert-sklearn' already exists and is not an empty directory.
Processing ./bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 131 kB 7.1 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 8.4 MB 42.2 MB/s 
     |████████████████████████████████| 138 kB 74.5 MB/s 
     |████████████████████████████████| 127 kB 60.6 MB/s 
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-py3-none-any.whl size=54247 sha256=5c30b89442cafbd137d14ebb149e4a24d713d87fd0a7b4f490a84261831bb537
  Stored in directory: /root/.cache/pip/whee

In [5]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import statistics as stats

from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

# **Import and load data**

In [6]:
ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/data mining/'
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/data mining/clear_text.xlsx'
TEST_PATH = "/content/drive/MyDrive/Colab Notebooks/data mining/clear_testtext.xlsx"

In [7]:
df = pd.read_excel(DATA_PATH)
df.tail()

,true_label,text,len,label
31517,__label__Do_an_va_do_uong,món kim chi ngon mùa thu ực ực ực chảy nước_mi...,40,3
31518,__label__Nha_dat,thuê chung_cư phạm văn đồng bắc từ_liêm hà_nội...,119,15
31519,__label__Kinh_doanh_va_Cong_nghiep,trình học_phí tặng khoá học kế_toán thuế trị_g...,91,9
31520,__label__Nha_va_vuon,bố_trí thông_minh giúp nhà_ống sài_gòn chỗ tối...,181,16
31521,__label__Kinh_doanh_va_Cong_nghiep,chúc_mừng sinh_nhật hoàng_gia media group tròn...,80,9


In [8]:
df_test=pd.read_excel(TEST_PATH)
df_test.tail()

,text,len
10012,tổng_hợp những món ngon khu_vực đào tấn ba_đìn...,173
10013,bản tin tài_chính kinh_doanh tối thứ_sáu tin t...,122
10014,ngang_nhiên vừa hack vừa game thủ bị ban ngay ...,32
10015,tour nước_ngoài dịp giáng_sinh năm mới giá tốt...,135
10016,phòng_không khép_kín ở ngõ xuân thuỷ giá đóng ...,59


# **Process data**

In [9]:
X, Y = df['text'], df['label']
X_test = df_test['text']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(df['text'], df['label'], random_state=42, test_size=0.2)

# **Navie Bayes + CountVectorizer**

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

In [12]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB())])

##**Tuning Parameter**

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
            #   'tfidf_max_df':[0.8, 0.9, 0.95],
            #   'tfidf_min_df': [0.1, 0.05, 0.15],
              'clf__alpha': (1e-2, 1e-3)}

In [15]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)


In [16]:
gs_clf.fit(X, Y)
#gs_clf.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': (0.01, 0.001),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2)]})

##Evaluate

In [ ]:
predicted = gs_clf.predict(X_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [ ]:
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(predicted, Y_test) * 100))
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder().fit(df.true_label)
true_labels = encoder.inverse_transform(range(23))
print(classification_report(Y_test, predicted, target_names=true_labels))

Accuracy: 95.21%
                                       precision    recall  f1-score   support

                   __label__Chinh_tri       1.00      1.00      1.00       294
         __label__Con_nguoi_va_xa_hoi       0.99      1.00      1.00       160
               __label__Cong_nghe_moi       1.00      1.00      1.00        14
            __label__Do_an_va_do_uong       1.00      1.00      1.00       961
                     __label__Du_lich       1.00      1.00      1.00       367
                    __label__Giai_tri       1.00      1.00      1.00        87
                    __label__Giao_duc       0.99      1.00      0.99       256
                  __label__Giao_thong       1.00      1.00      1.00        37
                    __label__Khoa_hoc       1.00      1.00      1.00        59
   __label__Kinh_doanh_va_Cong_nghiep       0.98      0.69      0.81       934
         __label__Lam_dep_va_the_hinh       1.00      1.00      1.00       111
 __label__Mang_internet_va_vien_th

#Testmodel

In [17]:
predicted = gs_clf.predict(X_test)

In [18]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder().fit(df.true_label)
true_labels = encoder.inverse_transform(predicted)

In [19]:
len(true_labels)

10017

In [20]:
with open("/content/drive/MyDrive/Colab Notebooks/data mining/predicttion/pred_naviebayes.txt", "wt") as f:
    for i in true_labels:
        f.write(i+"\n")